# tSNE collage

This notebook will show you how you can use a convolutional neural network (convnet) to search through a large collection of images. Specifically, it will show you how you can retrieve a set of images which are similar to a query image, returning you its `n` nearest neighbors in terms of image content.

In [ ]:
# Collage parameters

IMAGES_PATH = "source_images/"
TEST_IMAGE = IMAGES_PATH + "hedgehog_photos/palantir_retrospective_from_brian/american-river-tubing-2013_9719594465_o.jpg"

# The number of images to put in the collage. Will be sampled at random
# from all images in IMAGES_PATH. Try ~50–100 to make sure things are 
# working; ~1000 for a real collage.
MAX_NUM_IMAGES = 50

# The dimensions of the collage (measured in number of images/tiles)
COLLAGE_WIDTH  = 10
COLLAGE_HEIGHT = 5

if (COLLAGE_WIDTH * COLLAGE_HEIGHT != MAX_NUM_IMAGES):
    print("ERROR: COLLAGE_WIDTH * COLLAGE_HEIGHT must equal MAX_NUM_IMAGES (%d)." % MAX_NUM_IMAGES)

# The dimension of each tile. Aspect ratio should be 9 x 7
TILE_WIDTH = 216
TILE_HEIGHT = 168

In [ ]:
%matplotlib inline
import os
import random
import cPickle as pickle
import numpy as np
import matplotlib.pyplot
from matplotlib.pyplot import imshow
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from sklearn.decomposition import PCA
from scipy.spatial import distance
from tqdm import tqdm

# get_image(path) will handle the usual pre-processing steps: load an image 
# from our file system and turn it into an input vector of the correct 
# dimensions, those expected by VGG16, namely a color image of size 224x224.
#
# get_image will return a handle to the image itself, and a numpy array of 
# its pixels to input the network
def get_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

# Load a previously-trained neural network (VGG16) which comes with Keras.
# Other pre-trained networks here: https://keras.io/applications/
model = keras.applications.VGG16(weights='imagenet', include_top=True)
model.summary()

# Now we will remove the top classification layer from our network, leaving 
# the last fully-connected layer, "fc2 (Dense)" as the new output layer. The 
# way we do this is by instantiating a new model called feature_extractor 
# which takes a reference to the desired input and output layers in our VGG16 
# model. Thus, feature_extractor's output is the layer just before the 
# classification, the last 4096-neuron fully connected layer.
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)

# If we run the summary() function again, we see that the architecture of 
# feat_extractor is identical to the original model, except the last layer 
# has been removed. We also know that not just the architecture is the same, 
# but the two have the same weights as well.
feat_extractor.summary()

In [ ]:
# Next, we will load all of the images in a directory. This will search 
# recursively through all the folders in IMAGE_PATH. Set MAX_NUM_IMAGES to cap it at 
# some maximum number of images to load (it will grab a random subset of 
# MAX_NUM_IMAGES is less than the number of images in your directory.

IMAGES_PATH = "source_images/"

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(IMAGES_PATH) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
print("Found %d images." % len(images))

if MAX_NUM_IMAGES < len(images):
    images = [images[i] for i in sorted(random.sample(xrange(len(images)), MAX_NUM_IMAGES))]
print("Keeping %d images to analyze." % len(images))

In [ ]:
# Optionally, make sure things are working

def test_image(path):
    img, x = get_image(path)
    predictions = model.predict(x)
    imshow(img)
    for pred in decode_predictions(predictions)[0]:
        print("predicted %s with probability %0.3f" % (pred[1], pred[2]))
        
    # The predict function returns an array with one element per image (in 
    # our case, there is just one). Each element contains a 4096-element 
    # array, which is the activations of the last fully-connected layer in 
    # VGG16. Let's plot the array as well.
    feat = feat_extractor.predict(x)
    matplotlib.pyplot.figure(figsize=(16,4))
    matplotlib.pyplot.plot(feat[0])
    matplotlib.pyplot.show()
    
#test_image(TEST_IMAGE)

In [ ]:
# The next part will take the longest. We iterate through and extract the 
# features from all the images in our images array, placing them into an 
# array called features.
features = []
for image_path in tqdm(images):
    img, x = get_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)

In [ ]:
# Once that is done, we will take our nx4096 matrix of features (where 
# n is the number of images), and apply principal component analysis to 
# it, and keep the first 300 principal components, creating an nx300 
# matrix called pca_features.
features = np.array(features)
pca = PCA(n_components=300)
pca.fit(features)
pca_features = pca.transform(features)

# We are now ready to do our reverse image queries! The matrix 
# pca_features contains a compact representation of our images, one 
# 300-element row for each image with high-level feature detections. We 
# should expect that two similar images, which have similar content in 
# them, should have similar arrays in pca_features.

# Nearest neighbors

In [ ]:
# NOTE: should be cosine distance rather than Euclidean distance?

# Compute the euclidean distance between the PCA features of 
# query_image_idx-th image in our dataset, and the PCA features of 
# every image in the dataset (including itself, trivially 0). It then 
# returns an array of indices to the num_results (default is 5) most 
# similar images to it (not including itself).
def get_closest_images(query_image_idx, num_results=5):
    distances = [ distance.euclidean(pca_features[query_image_idx], feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[1:num_results+1]
    return idx_closest

def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
        img = image.load_img(images[idx])
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image

Finally we can do a query on a randomly selected image in our dataset.

In [ ]:
# do a query on a random image
query_image_idx = int(len(images) * random.random())
idx_closest = get_closest_images(query_image_idx)
query_image = get_concatenated_images([query_image_idx], 300)
results_image = get_concatenated_images(idx_closest, 200)

# display the query image
matplotlib.pyplot.figure(figsize = (5,5))
imshow(query_image)
matplotlib.pyplot.title("query image (%d)" % query_image_idx)

# display the resulting images
matplotlib.pyplot.figure(figsize = (16,12))
imshow(results_image)
matplotlib.pyplot.title("result images")

If we are satisfied with the quality of our image vectors, now would be a good time to save them to disk for later usage. You will need these vectors to run the [next notebook on making an image t-SNE](image-tsne.ipynb).

We need to save both the image features matrix (the PCA-reduced features, not the originals), as well as the array containing the paths to each image, to make sure we can line up the images to their corresponding vectors. 

In [ ]:
# Optionally, save off our image paths and PCA features
if False:
    pickle.dump([images, pca_features], open('interim/features_hedgehogs.p', 'wb'))

# t-SNE

In [ ]:
from PIL import Image
from sklearn.manifold import TSNE

# Optionally, load image paths and feature vectors from a saved file. 
if False:
    images, pca_features = pickle.load(open('interim/features_hedgehogs.p', 'r'))
    for i, f in zip(images, pca_features)[0:5]:
        # We can print their contents to get an idea of what they look like:
        print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(i, f[0], f[1], f[2], f[3]))
    
if len(images) > MAX_NUM_IMAGES:
    sort_order = sorted(random.sample(xrange(len(images)), MAX_NUM_IMAGES))
    images = [images[i] for i in sort_order]
    pca_features = [pca_features[i] for i in sort_order]

It is usually a good idea to first run the vectors through a faster dimensionality reduction technique like [principal component analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) to project your data into an intermediate lower-dimensional space before using t-SNE. This improves accuracy, and cuts down on runtime since PCA is more efficient than t-SNE. Since we have already projected our data down with PCA in the previous notebook, we can proceed straight to running the t-SNE on the feature vectors. Run the command in the following cell, taking note of the arguments:

- `n_components` is the number of dimensions to project down to. In principle it can be anything, but in practice t-SNE is almost always used to project to 2 or 3 dimensions for visualization purposes.
- `learning_rate` is the step size for iterations. You usually won't need to adjust this much, but your results may vary slightly. 
- `perplexity` refers to the number of independent clusters or zones t-SNE will attempt to fit points around. Again, it is relatively robust to large changes, and usually 20-50 works best. 
- `angle` controls the speed vs accuracy tradeoff. Lower angle means better accuracy but slower, although in practice, there is usually little improvement below a certain threshold.

In [ ]:
X = np.array(pca_features)
tsne = TSNE(n_components=2, learning_rate=150, perplexity=30, angle=0.2, verbose=2).fit_transform(X)

# Internally, t-SNE uses an iterative approach, making small (or sometimes 
# large) adjustments to the points. By default, t-SNE will go a maximum of 
# 1000 iterations, but in practice, it often terminates early because it 
# has found a locally optimal (good enough) embedding.
# 
# The variable `tsne` contains an array of unnormalized 2d points, 
# corresponding to the embedding. Here we normalize the embedding so that
# it lies entirely in the range (0,1).

tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [ ]:
# Finally, we will compose a new RGB image where the set of images have been 
# drawn according to the t-SNE results. Adjust `width` and `height` to set 
# the size in pixels of the full image, and set `max_dim` to the pixel size 
# (on the largest size) to scale images to.

width = 4000
height = 3000
max_dim = 100

full_image = Image.new('RGB', (width, height))
for img, x, y in tqdm(zip(images, tx, ty)):
    tile = Image.open(img)
    rs = max(1, tile.width/max_dim, tile.height/max_dim)
    tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)))

matplotlib.pyplot.figure(figsize = (16,12))
imshow(full_image)

In [ ]:
# Optionally, save t-SNE image
if False:
    full_image.save("output/hedgehogs_tSNE.jpg")

# Optionally, save the t-SNE points and their associated image paths 
# (for further processing in another environment).
if False:
    tsne_path = "../data/example-tSNE-points-hedgehogs.json"
    
    data = [{"path":os.path.abspath(img), "point":[x, y]} for img, x, y in zip(images, tx, ty)]
    with open(tsne_path, 'w') as outfile:
        json.dump(data, outfile)
    
    print("saved t-SNE result to %s" % tsne_path)

### Gridify with RasterFairy

In [ ]:
import rasterfairy

# We can optionally choose a grid size of rows (nx) and columns (ny), which 
# should be equal to the number of images you have. If it is less, then you 
# can simply cut the tsne and images lists to be equal to nx * ny.
# 
# If you omit the target=(nx, ny) argument, RasterFairy will automatically 
# choose an optimal grid size to be as square-shaped as possible. RasterFairy 
# also has options for embedding them in a grid with irregular borders as 
# well (see the GitHub page for more details).

# nx * ny = 1000, the number of images
nx = COLLAGE_WIDTH
ny = COLLAGE_HEIGHT

if (nx * ny != len(images)):
    raise Exception("nx * ny should equal %d." % len(images))

# assign to grid
grid_assignment = rasterfairy.transformPointCloud2D(tsne, target=(nx, ny))


# Set the tile_width and tile_height variables according to how big you want 
# the individual tile images to be. The resolution of the output image is 
# tile_width * nx x tile_height * ny. The script will automatically 
# center-crop all the tiles to match the aspect ratio of tile_width / 
# tile_height.

tile_width = TILE_WIDTH
tile_height = TILE_HEIGHT

full_width = tile_width * nx
full_height = tile_height * ny
aspect_ratio = float(tile_width) / tile_height

grid_image = Image.new('RGB', (full_width, full_height))

for img, grid_pos in tqdm(zip(images, grid_assignment[0])):
    idx_x, idx_y = grid_pos
    x, y = tile_width * idx_x, tile_height * idx_y
    tile = Image.open(img)
    tile_ar = float(tile.width) / tile.height  # center-crop the tile to match aspect_ratio
    if (tile_ar > aspect_ratio):
        margin = 0.5 * (tile.width - aspect_ratio * tile.height)
        tile = tile.crop((margin, 0, margin + aspect_ratio * tile.height, tile.height))
    else:
        margin = 0.5 * (tile.height - float(tile.width) / aspect_ratio)
        tile = tile.crop((0, margin, tile.width, margin + float(tile.width) / aspect_ratio))
    tile = tile.resize((tile_width, tile_height), Image.ANTIALIAS)
    grid_image.paste(tile, (int(x), int(y)))

matplotlib.pyplot.figure(figsize = (16,12))
imshow(grid_image)

In [ ]:
grid_image.save("output/hedgehogs_tSNE_grid.jpg")

# Shortest path

Another thing you can try is to do is fine a path between two images containing `n` images. The below is a naive approach to this problem which finds the closest image to the `n` vectors which are interpolated between those of the endpoint images. A better one would be to use a variant of [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) (implementation TBD). This implementation is not particularly good; improvement TBD (suggestions are welcome!)

With the naive approach, we run another principal component analysis, this time reducing down all the way to 3 dimensions. The reason for this is when there are too many dimensions and the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) sets in, most images cluster strongly around their class, and there are few images between classes.  In a low-dimensional space, this isn't as much a problem. So we first run a new PCA, saving the columns to `pca_features2`.

In [ ]:
features = np.array(features)
pca2 = PCA(n_components=3)
pca2.fit(features)
pca_features2 = pca2.transform(features)

Then we define our function `get_image_path_between` which will make `num_hops` sized stops between two images, and grab the closest image at each step, then concatenate them together and display them.

In [ ]:
def get_image_path_between(query_image_idx_1, query_image_idx_2, num_hops=4):
    path = [query_image_idx_1, query_image_idx_2]
    for hop in range(num_hops-1):
        t = float(hop+1) / num_hops
        lerp_acts = t * pca_features2[query_image_idx_1] + (1.0-t) * pca_features2[query_image_idx_2]
        distances = [distance.euclidean(lerp_acts, feat) for feat in pca_features2]
        idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])
        path.insert(1, [i for i in idx_closest if i not in path][0])
    return path

# pick image and number of hops
num_hops = 10
query_image_idx_1 = int(len(images) * random.random())
query_image_idx_2 = int(len(images) * random.random())

# get path
path = get_image_path_between(query_image_idx_1, query_image_idx_2, num_hops)

# draw image
path_image = get_concatenated_images(path, 200)
matplotlib.pyplot.figure(figsize = (36,12))
imshow(path_image)
matplotlib.pyplot.title("result images")